自分のGoogleアカウントと同期


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc2/jumanpp-2.0.0-rc2.tar.xz && \
tar xJvf jumanpp-2.0.0-rc2.tar.xz && \
rm jumanpp-2.0.0-rc2.tar.xz && \
cd jumanpp-2.0.0-rc2/ && \
mkdir bld && \
cd bld && \
cmake .. \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_INSTALL_PREFIX=/usr/local && \
make && \
sudo make install

In [ ]:
!mkdir -p /content/drive/'My Drive'/bert/japan_test

In [ ]:
cd /content/drive/'My Drive'/bert/japan_test

/content/drive/My Drive/bert/japan_test


In [ ]:
import urllib.request

kyoto_u_bert_url = "http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE.zip"
urllib.request.urlretrieve(kyoto_u_bert_url, "Japanese_L-12_H-768_A-12_E-30_BPE.zip")

('Japanese_L-12_H-768_A-12_E-30_BPE.zip',
 <http.client.HTTPMessage at 0x7f41ad6f00a0>)

In [ ]:
!unzip Japanese_L-12_H-768_A-12_E-30_BPE.zip

Archive:  Japanese_L-12_H-768_A-12_E-30_BPE.zip
replace Japanese_L-12_H-768_A-12_E-30_BPE/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!pip install transformers
!pip install pyknp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 KB 3.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, BertConfig
import numpy as np
from pyknp import Juman
jumanpp = Juman()

In [ ]:
config = BertConfig.from_json_file('/content/drive/My Drive/bert/japan_test/Japanese_L-12_H-768_A-12_E-30_BPE/bert_config.json')

model = BertForMaskedLM.from_pretrained('/content/drive/My Drive/bert/japan_test/Japanese_L-12_H-768_A-12_E-30_BPE/pytorch_model.bin', config=config)
bert_tokenizer = BertTokenizer('/content/drive/My Drive/bert/japan_test/Japanese_L-12_H-768_A-12_E-30_BPE/vocab.txt',
 do_lower_case=False, do_basic_tokenize=False)

Some weights of the model checkpoint at /content/drive/My Drive/bert/japan_test/Japanese_L-12_H-768_A-12_E-30_BPE/pytorch_model.bin were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
text = "患者が電子お薬手帳を夜に自宅でアプリから服薬指導を受ける"

result = jumanpp.analysis(text)
tokenized_text = [mrph.midasi for mrph in result.mrph_list()]
print(tokenized_text)

#tokenized_text[11] = tokenized_text[11] + tokenized_text[12]
#print(tokenized_text)

#tokenized_text.pop(12)
#tokenized_text.pop(3)
#tokenized_text.pop(3)
#print(tokenized_text)

['患者', 'が', '電子', 'お', '薬', '手帳', 'を', '夜', 'に', '自宅', 'で', 'アプリ', 'から', '服薬', '指導', 'を', '受ける']


In [ ]:
tokenized_text.insert(0, '[CLS]')
tokenized_text.append('[SEP]')
masked_index = 1
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([tokens])
model.eval()
with torch.no_grad():
  outputs = model(tokens_tensor)
  predictions = outputs[0]
_,predicted_indexes = torch.topk(predictions[0, masked_index], k=20)
predicted_tokens = bert_tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())
print(predicted_tokens)

['[CLS]', '[MASK]', 'が', '電子', 'お', '薬', '手帳', 'を', '夜', 'に', '自宅', 'で', 'アプリ', 'から', '服薬', '指導', 'を', '受ける', '[SEP]']
['母親', '父親', '主人公', '[UNK]', '先生', '医師', '女性', '母', '作者', '本人', '社長', '妻', '彼女', '男性', '親', '子供', '両親', '全員', '友人', '店長']


In [ ]:
path_w = '/content/drive/MyDrive/卒論研究/電子お薬手帳/step2/実験2/薬歴.txt'
with open(path_w, mode='w') as f:
    f.write('\n'.join(predicted_tokens))

# 新しいセクション

In [ ]:
text = "外国人が渋谷駅周辺経路誘導システムをオリンピック開催時に渋谷駅で利用する"
#text = "外国人観光客が渋谷駅周辺経路誘導システムをオリンピック開催時に渋谷駅で利用する"

result = jumanpp.analysis(text)
tokenized_text = [mrph.midasi for mrph in result.mrph_list()]
print(tokenized_text)

tokenized_text[0] = tokenized_text[0] + tokenized_text[1]
print(tokenized_text)

tokenized_text.pop(1)
#tokenized_text.pop(17)
print(tokenized_text)

['外国', '人', 'が', '渋谷', '駅', '周辺', '経路', '誘導', 'システム', 'を', 'オリンピック', '開催', '時', 'に', '渋谷', '駅', 'で', '利用', 'する']
['外国人', '人', 'が', '渋谷', '駅', '周辺', '経路', '誘導', 'システム', 'を', 'オリンピック', '開催', '時', 'に', '渋谷', '駅', 'で', '利用', 'する']
['外国人', 'が', '渋谷', '駅', '周辺', '経路', '誘導', 'システム', 'を', 'オリンピック', '開催', '時', 'に', '渋谷', '駅', 'で', '利用', 'する']


In [ ]:
tokenized_text.insert(0, '[CLS]')
tokenized_text.append('[SEP]')
masked_index = 17
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([tokens])
model.eval()
with torch.no_grad():
  outputs = model(tokens_tensor)
  predictions = outputs[0]
_,predicted_indexes = torch.topk(predictions[0, masked_index], k=20)
predicted_tokens = bert_tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())
print(predicted_tokens)

['[CLS]', '外国人', 'が', '渋谷', '駅', '周辺', '経路', '誘導', 'システム', 'を', 'オリンピック', '開催', '時', 'に', '渋谷', '駅', 'で', '[MASK]', 'する', '[SEP]']
['導入', '実施', '使用', '利用', '設置', '運用', '操作', '運行', '作動', '開始', '起動', '設定', '担当', '運転', '実証', '整備', '実行', '更新', '制御', '接続']


In [ ]:
path_w = '/content/drive/MyDrive/kenkyu/step2/実験2/外国人-利用する.txt'
with open(path_w, mode='w') as f:
    f.write('\n'.join(predicted_tokens))